# PACKAGE

In [ ]:
import pandas as pd
import numpy as np
import cv2
import math
import os
import re
import warnings
from copy import deepcopy
from PIL import Image
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
from matplotlib import pyplot as plt
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
warnings.filterwarnings("ignore")
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import re
import xml.etree.ElementTree as ET
import os
import copy
import sys
import argparse
import shutil
import collections
import subprocess
from IPython.core.display import HTML

# READ

In [ ]:
MODEL_CLASSIFIER = 'lgbm_20211223'

In [ ]:
EXCLUDED_BLUE = True

In [ ]:
label2id = {
    'dát_tăng_sắc_tố_(vết_thâm)': 0,
    'sang_thương_viêm_(sẩn,_mụn_mủ,_mảng_viêm_đỏ)': 1,
    'sẹo_mụn_(lõm,_lồi)': 2,
    'còi_(đóng/mở)': 3,
    'sang_thương_nang_và_nốt': 4,
}

In [ ]:
id2label = {
    0: 'vet_tham',
    1: 'thuong_viem',
    2: 'seo_mun',
    3: 'coi',
    4: 'thuong_nang'
}
id2label

In [ ]:
# Float
id2color = {
    0: (0.1, 0.1, 0.7), #blue-black
    1: (1.0,0.5,1.0), #pink
    2: (0, 0.6, 0), #green
    3: (0.4, 0.9, 0.9), #light blue
    4: (0.9, 0, 0) #red
}

In [ ]:
def object_check(x):
    if os.path.isdir(x):
        return False
    with open(x, 'r') as f:
        lines = f.readlines()
        if (len(lines) == 0):
            return False
    return True

In [ ]:
img_ids = os.listdir('/hdd2/skin/app_data/bbox/')
d = pd.DataFrame({'id': img_ids})
d['coco'] = d['id'].apply(lambda x: os.path.join('/hdd2/skin/app_data/bbox/', x))
d['coco_check'] = d['coco'].apply(lambda x: os.path.exists(x))
d['image'] = d['id'].apply(lambda x: os.path.join('/hdd2/skin/app_data/bbox_img/', x + '.jpg'))
d['image_check'] = d['image'].apply(lambda x: os.path.exists(x))
d['object_check'] = d['coco'].apply(lambda x: object_check(x))

In [ ]:
len(img_ids), img_ids[:3]

In [ ]:
d

In [ ]:
# os.listdir('/hdd2/skin/app_data/bbox_img/')[:3]

In [ ]:
d[['coco_check', 'image_check', 'object_check']].sum()

In [ ]:
# len(os.listdir('/hdd2/skin/app_data/bbox_img'))

# len(os.listdir('/hdd2/skin/app_data/bbox_img'))

# len(os.listdir('/hdd2/skin/app_data/data1/images_1'))

# len(os.listdir('/hdd2/skin/app_data/data2/images'))

# len(os.listdir('/hdd2/skin/app_data/CIRCLE_01/images_bbox_1'))

# len(os.listdir('/hdd2/skin/app_data/CIRCLE_02/images_bbox_2'))

In [ ]:
d = d[(d['coco_check'] == True) & (d['image_check'] == True) & (d['object_check'] == True)]
d = d.reset_index(drop=True)

In [ ]:
d

In [ ]:
grad_d = pd.read_csv('grade.xls', index_col=0)
grad_d = grad_d.rename(columns={'Grade': 'grading', 'ID': 'id'})
grad_d = grad_d[grad_d['grading'].isin(['0','1','2','3','4'])]
grad_d

In [ ]:
grad_d['grading'].value_counts()

In [ ]:
d = d.merge(grad_d[['id', 'grading']], left_on='id', right_on='id')

In [ ]:
d

In [ ]:
d['grading'].value_counts()

## INSPECT

In [ ]:
def plot_img(img):
    plt.figure(figsize=(7, 7))
    plt.imshow(img)
    plt.show()

In [ ]:
def draw_bounding_boxes(img_id, plot_im=True):
    img_path = d[d['id'] == img_id]['image'].tolist()[0]
    bbox_tuple = read_coco(img_id)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_copy = deepcopy(img)
    labels, x0s, y0s, x1s, y1s = bbox_tuple
    for i, (x0, y0, x1, y1) in enumerate(zip(x0s, y0s, x1s, y1s)):
        cv2.rectangle(img_copy, (x0, y0), (x1, y1), (0, 0, 255), 5)
    if plot_im:
        plot_img(img_copy)
    return img

In [ ]:
def read_content(file_content):
    lines = file_content
    labels, x0s, y0s, x1s, y1s = [], [], [], [], []
    for line in lines:
        line = line.strip()
        elements = line.split(' ')
        label, x0, y0, x1, y1 = elements
        x0 = int(x0)
        y0 = int(y0)
        x1 = int(x1)
        y1 = int(y1)
        labels.append(label)
        x0s.append(x0)
        y0s.append(y0)
        x1s.append(x1)
        y1s.append(y1)
    return (labels, x0s, y0s, x1s, y1s)

In [ ]:
def read_coco(img_id):
    path = d[d['id'] == img_id]['coco'].tolist()[0]
    with open(path, 'r') as f:
        content = f.readlines()
        return read_content(content)

In [ ]:
# img = draw_bounding_boxes('ckls4ugoq00gq3a68u5ddifuw')

## DATASET

In [ ]:
class AppDataset(Dataset):
    
    def __init__(self, d, transforms=None):
        super().__init__()
        
        self.image_ids = d["id"].unique()
        self.df = d
        self.transforms = transforms
        
    def __getitem__(self, index):
        image_id = self.image_ids[index]
        labels, x0s, y0s, x1s, y1s = read_coco(image_id)
        
        image = cv2.imread(self.df[self.df['id'] == image_id]['image'].tolist()[0])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = image/255.0
        image = image.transpose(2,0,1)
        image = torch.from_numpy(image).float()
       
        boxes = np.array(list(zip(x0s, y0s, x1s, y1s)))
        if boxes.shape == (0,):
            print(index, len(self.df), image_id, boxes.shape, labels, x0s, y0s, x1s, y1s)
            raise Exception('Wrong format')
        labels = torch.tensor([label2id[l] for l in labels], dtype=torch.int64)
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        num_objs = len(boxes)
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target['boxes'] = torch.from_numpy(boxes)
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])
        target["area"] = torch.from_numpy(area)
        target["iscrowd"] = iscrowd
        return image, target
    
    def __len__(self):
        return self.image_ids.shape[0]


In [ ]:
def get_valid_transform():
    return A.Compose([
        A.Normalize(mean=(0, 0, 0), std=(1, 1, 1), max_pixel_value=255.0, p=1.0),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
def mAP(test_dataset, test_data_loader, threshold=0.3):
    it_data_loader = iter(test_data_loader)
    det_image_ids = list()
    det_boxes = list()
    det_labels = list()
    det_scores = list()
    true_boxes = list()
    true_labels = list()
    true_difficulties = list()
    idx = 0
    for i in range(len(test_dataset)):
        images, targets = next(it_data_loader)
        images = list(img.to(device) for img in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        image_ids = [k['image_id'] for k in targets]
        preds = model(images)
        preds = [{k: v.to(cpu) for k, v in t.items()} for t in preds]
        boxes = targets[idx]['boxes'].cpu().numpy().astype(np.int32)
        image = images[idx].cpu().numpy()
        image = image.transpose(1,2,0)
        pred_scores, pred_boxes, pred_classes = get_prediction(preds[idx], threshold)
        pred_boxes = pred_boxes.astype(np.int32)
        det_boxes_batch = torch.from_numpy(pred_boxes)
        det_labels_batch = torch.from_numpy(pred_classes)
        det_scores_batch = torch.from_numpy(pred_scores)
        boxes = targets[idx]['boxes']
        labels = targets[idx]['labels']
        det_boxes.append(det_boxes_batch)
        det_labels.append(det_labels_batch)
        det_scores.append(det_scores_batch)
        true_boxes.append(boxes)
        true_labels.append(labels)
        det_image_ids.extend(image_ids)
    copy2voc(test_dataset, det_image_ids, true_labels, true_boxes, det_labels, det_scores, det_boxes)
    result = subprocess.run(["python", "pascalvoc.py", "-t", "0.1"], cwd="/hdd2/skin/app_data/lab1/mAP", capture_output=True)
    return result

In [ ]:
def copy2voc(selected_dataset, det_image_ids, true_labels, true_boxes, det_labels, det_scores, det_boxes):
    shutil.rmtree('/hdd2/skin/app_data/lab1/mAP/detections')
    shutil.rmtree('/hdd2/skin/app_data/lab1/mAP/groundtruths')
    shutil.rmtree('/hdd2/skin/app_data/lab1/mAP/results')
    os.makedirs('/hdd2/skin/app_data/lab1/mAP/detections')
    os.makedirs('/hdd2/skin/app_data/lab1/mAP/groundtruths')
    os.makedirs('/hdd2/skin/app_data/lab1/mAP/results')

    for i, tl, tb, dl, ds, db in zip(det_image_ids, true_labels, true_boxes, det_labels, det_scores, det_boxes):
        idx = i.cpu().numpy()[0]
        fn = selected_dataset.image_ids[idx] + '.txt'
        # Write groundtruth content
        gt_fn = os.path.join('/hdd2/skin/app_data/lab1/mAP/groundtruths', fn)
        with open(gt_fn, 'w') as gt_file:
            lines = []
            for j in range(len(tl)):
                _id = tl[j].cpu().numpy().tolist()
                if _id != 0 or EXCLUDED_BLUE is False:
                    label = id2label[_id]
                    x0 = tb[j].cpu().numpy()[0]
                    y0 = tb[j].cpu().numpy()[1]
                    x1 = tb[j].cpu().numpy()[2]
                    y1 = tb[j].cpu().numpy()[3]
                    line = f'{label} {x0} {y0} {x1} {y1}'
                    lines.append(line)
            content = '\n'.join(lines)
            gt_file.writelines(content)

        # Write detection content
        dt_fn = os.path.join('/hdd2/skin/app_data/lab1/mAP/detections', fn)
        with open(dt_fn, 'w') as dt_file:
            lines = []
            for j in range(len(dl)):
                label = id2label[dl[j].cpu().numpy().tolist()]
                score = ds[j].cpu().numpy()
                x0 = db[j].cpu().numpy()[0]
                y0 = db[j].cpu().numpy()[1]
                x1 = db[j].cpu().numpy()[2]
                y1 = db[j].cpu().numpy()[3]
                line = f'{label} {score} {x0} {y0} {x1} {y1}'
                lines.append(line)
            content = '\n'.join(lines)
            dt_file.writelines(content)

In [ ]:
test_dataset = AppDataset(d, get_valid_transform())
test_data_loader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

In [ ]:
cpu = torch.device('cpu')
device = torch.device('cuda:1')
cpu, device

### MODEL

In [ ]:
model = torch.load('../model_FRCNN50/resnet50_epoch12700.pkl', map_location="cuda:1")

In [ ]:
NUM_CLASSES = len(label2id)
NUM_CLASSES

### EVALUATE

In [ ]:
model.eval();

In [ ]:
def get_prediction(pred_dict, threshold=0.3, topk=3):
    pred_class = pred_dict['labels'].detach().numpy()
    pred_boxes = pred_dict['boxes'].detach().numpy()
    pred_score = pred_dict['scores'].detach().numpy()
    pred_t = pred_score >= threshold
    pred_boxes = pred_boxes[pred_t]
    pred_class = pred_class[pred_t]
    pred_scores = pred_score[pred_t]
    return pred_scores, pred_boxes, pred_class

In [ ]:
def plot_with_bbox(img, gt_boxes, gt_classes, pred_boxes, pred_class, plot_bbox=False, plot_class=True):
    linewidth = img.shape[0]//500
    if plot_bbox:
        img_copy1 = deepcopy(img)
        img_copy2 = deepcopy(img)
        fig, ax = plt.subplots(1, 1, figsize=(24, 14))
        for box, _class in zip(gt_boxes, gt_classes):
            cv2.rectangle(img_copy1, (box[0], box[1]), (box[2], box[3]), (0, 255, 255), linewidth)
        for box, _class in zip(pred_boxes, pred_class):
            cv2.rectangle(img_copy2, (box[0], box[1]), (box[2], box[3]), (255, 0, 255), linewidth)
        hstack_np = np.hstack((img, img_copy1, img_copy2))
        ax.imshow(hstack_np)
    if plot_class:
        img_copy1 = deepcopy(img)
        img_copy2 = deepcopy(img)
        fig, ax = plt.subplots(1, 1, figsize=(24, 14))
        for box, _class in zip(pred_boxes, pred_class):
            cv2.rectangle(img_copy2, (box[0], box[1]), (box[2], box[3]), id2color[_class], linewidth)
        for box, _class in zip(gt_boxes, gt_classes):
            if _class != 0 or EXCLUDED_BLUE is False:
                cv2.rectangle(img_copy1, (box[0], box[1]), (box[2], box[3]), id2color[_class], linewidth)
        hstack_np = np.hstack((img, img_copy1, img_copy2))
        ax.imshow(hstack_np)
    plt.show()

In [ ]:
result = mAP(test_dataset, test_data_loader)

In [ ]:
print(result.stdout.decode('UTF-8'))

## GRADING

In [ ]:
# def feat_eng(bbox_tuple):
#     acnes = bbox_tuple[0]
#     counter = collections.Counter(acnes)
#     feats = counter
#     feat = np.zeros(len(label2id))
#     for i in range(len(label2id)):
#         feat[i] = feats.get(i,0)
#     feat_agg = feat.tolist()
#     return feat_agg

# def get_grading_cls(pred_boxes, pred_class):
#     import joblib
#     from_file = joblib.load(MODEL_CLASSIFIER)
#     feat_agg = feat_eng((pred_class, pred_boxes[:,0],pred_boxes[:,1],pred_boxes[:,2],pred_boxes[:,3]))
#     grading = from_file.predict(np.array(feat_agg).reshape(1, -1))
#     return grading

In [ ]:
def get_grading_cls(pred_boxes, pred_class):
    counter = collections.Counter(pred_class)
    grading = 0
    
    post_inflammatory = counter.get(0, 0)
    inflammatory_lesion = counter.get(1, 0)
    acne_scar = counter.get(2, 0)
    black_whitehead = counter.get(3, 0)
    nodular_lesion = counter.get(4, 0)
    
    if nodular_lesion >= 2:
        grading = 4
    elif nodular_lesion == 1:
        grading = 3
    elif inflammatory_lesion >= 2:
        grading = 2
    elif (post_inflammatory + acne_scar + black_whitehead >= 10) or (inflammatory_lesion == 1):
        grading = 1
    elif post_inflammatory + acne_scar + black_whitehead < 10:
        grading = 0
    else:
        grading = 0
    return grading

In [ ]:
import collections

In [ ]:
export = []

In [ ]:
idx=0
test_df = d
it_data_loader = iter(test_data_loader)
y_true, y_pred = [], []
for i in range(len(test_df[:])):
    true_grade = int(test_df['grading'].tolist()[i])
    try:
        img_id = test_df['id'].tolist()[i]
        images, targets = next(it_data_loader)
        images = list(img.to(device) for img in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        gt_classes = targets[0]['labels'].cpu().numpy().astype(np.int32)
        preds = model(images)
        preds = [{k: v.to(cpu) for k, v in t.items()} for t in preds]

        ## Groundtruth
        boxes = targets[idx]['boxes'].cpu().numpy().astype(np.int32)
        sample = images[idx].cpu().numpy()
        sample = sample.transpose(1,2,0)

        # Pred
        pred_scores, pred_boxes, pred_class = get_prediction(preds[idx], threshold=0.2)
        pred_boxes = pred_boxes.astype(np.int32)

        # Fix
        pred_grade = get_grading_cls(pred_boxes, pred_class)
        y_true.append(true_grade)
        y_pred.append(pred_grade)

        gt_classes_c = collections.Counter(gt_classes)
        pred_class_c = collections.Counter(pred_class)
        levels =list(range(0,5))
        trues, preds = [], []
        for i in levels:
            trues.append(gt_classes_c.get(i, 0))
            preds.append(pred_class_c.get(i,0))
        c_report = pd.DataFrame({
            'level': levels,
            'true': trues,
            'pred': preds
        })
        export.append([img_id]+ trues + preds)
        

#         display(HTML(f'<h1>True Grade: {true_grade} &emsp; Pred Grade: {pred_grade}</h1>'))
#         display(c_report)
#         plot_with_bbox(sample, boxes, gt_classes, pred_boxes, pred_class)
#         print('======================================================================================================================================================================================')


    except StopIteration:
        pass


In [ ]:
export_df = pd.DataFrame(columns=['img_id','t_0','t_1','t_2','t_3','t_4','p_0','p_1','p_2','p_3','p_4'], data=export)

In [ ]:
export_df

In [ ]:
export_df.to_csv('prediction.csv')

In [ ]:
from sklearn.utils.multiclass import unique_labels
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
len(y_true)

In [ ]:
y_pred = [int(i) for i in y_pred]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))

In [ ]:
plot_confusion_matrix(y_true, y_pred, classes=[0,1,2,3,4])

In [ ]:
plot_confusion_matrix(y_true, y_pred, classes=[0,1,2,3,4], normalize=True)